# Hperparameter optimization with Optuna

In [4]:
import optuna

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Keras Imports
from keras import backend
from keras.layers.core import Activation
from keras.callbacks import EarlyStopping

from keras.losses import MeanSquaredError

import os


2023-02-26 18:16:26.366874: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 18:16:26.566309: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-26 18:16:26.583184: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/humble/opt/rviz_ogre_vendor/lib:/opt/ros/humble/lib/x86_64-linux-gnu:/opt/

In [5]:
# Data constants
DATASET_ID = 1
DATA_PATH = f"/home/feline/Documents/KIT/WS22_23/LAMA/cassy/belle_neural_trigger/notebooks/data/random{DATASET_ID}.csv"
# DATA_PATH = f"{os.getcwd()}/belle_neural_trigger/data/random{DATASET_ID}.csv"
DATA_COUNT = 500

# Random seed
RANDOM_SEED = 42

METRIC = "accuracy"


# mlp setup constants
MIN_HIDDENLAYERS = 1
MAX_HIDDENLAYERS = 10

MIN_NEURON_PER_LAYER = 1
MAX_NEURON_PER_LAYER = 300

WEIGHT_THRESH = 0.05,

NUM_EPOCHS = 500


#Training
N_TRAILS = 20
N_TRAILS_2 = 50

In [7]:
data = pd.read_csv(DATA_PATH, delimiter='\t');    
data = data.iloc[:DATA_COUNT, 9:]

input = data.iloc[:, :-9]
target = data.iloc[:, -9:-7]

In [8]:
input.head()

,SL0-relID,SL0-driftT,SL0-alpha,SL1-relID,SL1-driftT,SL1-alpha,SL2-relID,SL2-driftT,SL2-alpha,SL3-relID,...,SL5-alpha,SL6-relID,SL6-driftT,SL6-alpha,SL7-relID,SL7-driftT,SL7-alpha,SL8-relID,SL8-driftT,SL8-alpha
26,0.000000,0.019646,0.518803,-0.175781,0.030929,0.418625,0.0,0.039879,0.265662,-0.140625,...,0.000000,0.0,0.000000,-0.497786,-0.929688,0.094763,0.483770,0.000,0.107461,0.000216
26,0.000000,0.019646,0.518803,-0.175781,0.030929,0.418625,0.0,0.039879,0.265662,-0.140625,...,0.000000,0.0,0.000000,-0.497786,-0.929688,0.094763,0.483770,0.000,0.107461,0.000216
26,0.003906,-0.043702,0.372666,1.000000,-0.068906,0.000000,0.0,0.000000,-0.307539,-0.015625,...,0.608852,0.0,-0.189303,-0.187435,0.000000,-0.215980,0.319851,0.125,-0.242333,0.000308
26,0.003906,-0.043702,0.372666,1.000000,-0.068906,0.000000,0.0,0.000000,-0.307539,-0.015625,...,0.608852,0.0,-0.189303,-0.187435,0.000000,-0.215980,0.319851,0.125,-0.242333,0.000308
26,0.003906,-0.041512,0.166792,1.000000,-0.065440,0.049572,0.0,-0.088371,-0.554250,-0.015625,...,0.113390,0.0,-0.179397,-0.301857,0.000000,-0.204515,0.119405,0.125,-0.229258,0.000308


In [9]:
target.head()

,RecoZ,RecoTheta
26,0.065285,0
26,0.065285,0
26,0.080510,0
26,0.080510,0
26,0.080510,0


In [10]:
# activation function: tanh(x/2)
def tanh(x):
    return backend.tanh(x/2)

In [11]:
def bias_(model):
    for layer in model.layers:
            if isinstance(layer, tf.keras.layers.Dense):
                weights = layer.get_weights()
                
                weight = weights[0]
                bias = weights[1]
                print(f"weights_")
                print(weight)
                print(bias)
            
                #bias_values.append(np.sum(np.abs(bias)))
                #weight_values.append(np.sum(np.abs(weight)))

                weight_threshold = np.mean(np.abs(weight)) * 0.1
                bias_threshold = np.mean(np.abs(bias)) * 0.05

                amount_dead_weights = len(np.where(weight < weight_threshold)[0])
                amount_dead_bias = len(np.where(bias< weight_threshold)[0])



In [12]:
def mlp_arcitecture(num_neurons_per_layer):
    num_of_neurons = sum(num_neurons_per_layer)
    num_neurons_per_layer = np.array(num_neurons_per_layer)
    
    print(f'------numbers of neuron per layer-----')
    print(num_neurons_per_layer)
    print(num_neurons_per_layer[0])
        
    print(num_of_neurons)
    print(f'------')

In [13]:
def build_model(num_neurons_per_layer):
    # activation function
    activation_function = Activation(tanh)

     # create model: 
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(num_neurons_per_layer[0],
            input_shape = (27,),  
            activation=activation_function)
    ])

    for layer in range(1, len(num_neurons_per_layer)):
        #num_neurons = int(num_neurons_per_layer[layer])
        model.add(
            tf.keras.layers.Dense(
                num_neurons_per_layer[layer],
                activation = activation_function))

    model.add(tf.keras.layers.Dense(
        2, 
        activation = activation_function
        ))
    
    # Specify the loss fuction, optimizer, metrics
    model.compile(
        loss = tf.keras.losses.MeanSquaredError(),
        optimizer = "sgd",
        metrics = [METRIC]
    )
    return model
    

In [14]:
def mlp_weights(model):
    weights = model.get_weights()
    amount_dead_weights = 0
    for layer in model.layers:
        weights = layer.get_weights()
        amount_dead_weights = amount_dead_weights + len(np.where(abs(weights[0]) < WEIGHT_THRESH)[0])
    return amount_dead_weights

In [15]:
# Define the objective function, with on hyperparameter
def objective(trial):

    number_of_layer = trial.suggest_int("number_of_layer", MIN_HIDDENLAYERS, MAX_HIDDENLAYERS)
    num_neurons_per_layer = []

    for i in range(number_of_layer):
        layer_ = trial.suggest_int('layer_{}'.format(i), MIN_NEURON_PER_LAYER, MAX_NEURON_PER_LAYER)
        num_neurons_per_layer.append(layer_)
    
    #sum_neuron =   mlp_arcitecture(num_neurons_per_layer)
    num_neurons_per_layer = np.array(num_neurons_per_layer)
    
    
    model = build_model(num_neurons_per_layer)

    # add early stopping callback to save time
    es = EarlyStopping(monitor = f'val_{METRIC}', mode ='max', patience = 5)
    
    # Train the model
    history = model.fit(input, target, epochs= NUM_EPOCHS, validation_split = 0.2, batch_size= 2, callbacks = [es], use_multiprocessing=True, verbose=0)
    
   #print(f'amount of dead weight{amount_dead_weights} and amount of dead bias {amount_dead_bias}')
    last_result = history.history[f"val_{METRIC}"][-1:][0]


    return last_result

In [16]:
# study with objectiv with optimizes only the accuracy
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials= N_TRAILS)



# Print the best hyperparameters and corresponding accuracy
print('Best hyperparameters: {}'.format(study.best_params))
print('Best accuracy: {}'.format(study.best_value))


[I 2023-02-26 18:17:23,501] A new study created in memory with name: no-name-b4206ef6-675c-4a54-a71e-ea763c97e361


-----------


2023-02-26 18:17:23.874165: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/humble/opt/rviz_ogre_vendor/lib:/opt/ros/humble/lib/x86_64-linux-gnu:/opt/ros/humble/lib
2023-02-26 18:17:23.875745: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-26 18:17:23.876754: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (feline-Yoga-7-15ITL5): /proc/driver/nvidia/version does not exist
2023-02-26 18:17:23.891410: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in o

Best hyperparameters: {'number_of_layer': 2, 'layer_0': 266, 'layer_1': 184}
Best accuracy: 0.9100000262260437
----------j-


In [23]:

fig_num_of_layer = optuna.visualization.plot_slice(study, ["number_of_layer"],target_name=METRIC)
fig_all_layer = optuna.visualization.plot_slice(study,target_name=METRIC)
fig_num_of_layer.show()
fig_all_layer.show()


In [19]:
fig_number_of_layer= optuna.visualization.plot_parallel_coordinate(study,params= ["number_of_layer"],target_name=METRIC)
fig_number_of_layer.show()
fig_layer_conection = optuna.visualization.plot_parallel_coordinate(study,params= ["layer_0","layer_1","number_of_layer"],target_name=METRIC)
fig_layer_conection.show()

In [20]:
fig = optuna.visualization.plot_optimization_history(study,target_name=METRIC)
fig.show()

In [27]:
# Define the objective function, multi hyperparameter optimation
def objective_(trial):

    number_of_layer = trial.suggest_int("number_of_layer", 1, 5)
    num_neurons_per_layer = []

    for i in range(number_of_layer):
        layer_ = trial.suggest_int('layer_{}'.format(i), 1, 20)
        num_neurons_per_layer.append(layer_)
    
    num_of_neurons = sum(num_neurons_per_layer)
    num_neurons_per_layer = np.array(num_neurons_per_layer)
    
    
    model = build_model(num_neurons_per_layer)

    # add early stopping callback to save time
    es = EarlyStopping(monitor = f'val_{METRIC}', mode ='max', patience = 5)
    
    # Train the model
    history = model.fit(input, target, epochs= NUM_EPOCHS, validation_split = 0.2, batch_size= 2, callbacks = [es], use_multiprocessing=True, verbose=0)

    amount_dead_weights = mlp_weights(model)
    
   #print(f'amount of dead weight{amount_dead_weights} and amount of dead bias {amount_dead_bias}')
    last_result = history.history[f"val_{METRIC}"][-1:][0]

    return last_result, amount_dead_weights, num_of_neurons

In [29]:
study_2 = optuna.create_study(directions=["maximize", "minimize","minimize"])
study_2.optimize(objective_, n_trials=20)


[I 2023-02-26 18:27:49,735] A new study created in memory with name: no-name-c2c6216f-5f99-48f0-a9aa-357350cf0890
[I 2023-02-26 18:27:51,156] Trial 0 finished with values: [0.6700000166893005, 70.0, 46.0] and parameters: {'number_of_layer': 5, 'layer_0': 11, 'layer_1': 3, 'layer_2': 9, 'layer_3': 16, 'layer_4': 7}. 
[I 2023-02-26 18:27:55,382] Trial 1 finished with values: [0.8899999856948853, 31.0, 9.0] and parameters: {'number_of_layer': 1, 'layer_0': 9}. 
[I 2023-02-26 18:27:56,640] Trial 2 finished with values: [0.6700000166893005, 40.0, 24.0] and parameters: {'number_of_layer': 3, 'layer_0': 9, 'layer_1': 14, 'layer_2': 1}. 
[I 2023-02-26 18:27:58,794] Trial 3 finished with values: [0.8700000047683716, 30.0, 9.0] and parameters: {'number_of_layer': 1, 'layer_0': 9}. 
[I 2023-02-26 18:28:00,027] Trial 4 finished with values: [0.6700000166893005, 14.0, 19.0] and parameters: {'number_of_layer': 3, 'layer_0': 3, 'layer_1': 1, 'layer_2': 15}. 
[I 2023-02-26 18:28:01,772] Trial 5 finish

In [30]:

trial_with_highest_accuracy = max(study_2.best_trials, key=lambda t: t.values[1])
print(f"Trial with highest accuracy: ")
print(f"\tnumber: {trial_with_highest_accuracy.number}")
print(f"\tparams: {trial_with_highest_accuracy.params}")
print(f"\tvalues: {trial_with_highest_accuracy.values}")

Trial with highest accuracy: 
	number: 10
	params: {'number_of_layer': 1, 'layer_0': 5}
	values: [0.8999999761581421, 17.0, 5.0]


In [38]:
fig_1 = optuna.visualization.plot_pareto_front(study_2, target_names=[METRIC,"dead_weights","num_neuron"])
fig_1.show()


In [44]:
hyperparameter_importances = optuna.visualization.plot_param_importances(
    study_2, target=lambda t: t.values[0], target_name="accuracy"
)
hyperparameter_importances.show()
